<a href="https://colab.research.google.com/github/GerardoQuizhpe/Detecci-n-de-cascos-de-seguridad-con-YOLOv8n/blob/main/Detecci%C3%B3n_de_cascos_YOLOv8n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Detección de cascos de seguridad usando YOLOv8n**
### **Nombre:** Gerardo Quizhpe

**Requerimientos**

**Instalar librerias necesarias para usar el modelo**

In [ ]:
!pip install ultralyticsplus==0.0.24 ultralytics==8.0.23 opencv-python gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

**Importar modelo**

In [ ]:
from ultralyticsplus import YOLO, render_result
import matplotlib.pyplot as plt

# load model
model = YOLO('keremberke/yolov8n-hard-hat-detection')

# set model parameters
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections image

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/6.23M [00:00<?, ?B/s]

**Funciones para procesar las imágenes y video**

In [ ]:
# Función para procesar imágenes
def process_image(image):
    # Convertir la imagen de BGR a RGB (ya que YOLO espera RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Realizar la inferencia
    results = model.predict(image_rgb)

    # Convertir el resultado a una imagen
    render = render_result(model=model, image=image_rgb, result=results[0])
    render_np = np.array(render)

    # Convertir la imagen renderizada de vuelta a BGR para Gradio
    render_bgr = cv2.cvtColor(render_np, cv2.COLOR_RGB2BGR)

    return render_bgr

# Función para procesar videos
def process_video(video_path):
    # Capturar el video
    cap = cv2.VideoCapture(video_path)

    # Obtener el ancho y alto del video
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Definir el codec y crear el objeto VideoWriter para guardar el video procesado
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para el video de salida
    out_path = '/content/output_video.mp4'
    out = cv2.VideoWriter(out_path, fourcc, 20.0, (width, height))

    # Procesar cada fotograma del video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir el fotograma de BGR a RGB (ya que YOLO espera RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Realizar la inferencia
        results = model.predict(frame_rgb)

        # Convertir el resultado a una imagen
        render = render_result(model=model, image=frame_rgb, result=results[0])
        render_np = np.array(render)

        # Convertir la imagen renderizada de vuelta a BGR para OpenCV
        render_bgr = cv2.cvtColor(render_np, cv2.COLOR_RGB2BGR)

        # Escribir el fotograma procesado en el archivo de salida
        out.write(render_bgr)

    # Liberar el objeto de captura y el objeto de escritura
    cap.release()
    out.release()

    return out_path

**Probar modelo**

In [ ]:
import gradio as gr
import cv2
import numpy as np

**Crear interfaz con Gradio para subir imágenes**

In [ ]:
# Crear la interfaz de Gradio para imágenes
image_interface = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="numpy", label="Subir Imagen"),
    outputs=gr.Image(type="numpy", label="Resultado Imagen"),
    live=True,
    title="Detección de Cascos en Imágenes",
    description="Sube una imagen para detectar"
)

# Ejecutar interfaz
image_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://18fbb4368badd0a9af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


**Crear interfaz con Gradio para subir videos**

In [ ]:
# Crear la interfaz de Gradio para videos
video_interface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(label="Subir Video"),
    outputs=gr.Video(label="Resultado Video"),
    live=True,
    title="Detección de Cascos en Videos",
    description="Sube un video para detectar"
)

# Ejecutar interfaz
video_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bb4402ce8408de213a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
